In [1]:
import numpy as np
from utils import *
import random

In [2]:
data = open('datasets/dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

print("There are %d total characters and %d unique characters in your data." % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
char_to_ix = {ch:i for i, ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i, ch in enumerate(sorted(chars))}

print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients["dWaa"], gradients["dWax"], gradients["dWya"], gradients["db"], gradients["dby"]
    
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, 'db': db, 'dby':dby}
    return gradients

In [5]:
np.random.seed(3)
dWax = np.random.randn(5, 3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2, 5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1) * 10
gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, 'db': db, 'dby':dby}

gradients = clip(gradients, 10)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]


In [6]:
def sample(parameters, char_to_ix, seed):
    Waa, Wya, Wax, b, by = parameters["Waa"], parameters["Wya"], parameters["Wax"], parameters["b"], parameters["by"]
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    n_x = Wax.shape[1]
    
    x = np.zeros((n_x, 1))
    a_prev = np.zeros((n_a, 1))
    
    indices = []
    
    idx = -1
    counter = 0
    newline_character = char_to_ix["\n"]
    
    while (idx != newline_character and counter != 50):
        a = np.tanh(np.dot(Wax, x)+np.dot(Waa, a_prev)+b)
        z = np.dot(Wya, a)+by
        y = softmax(z)
        
        np.random.seed(counter+seed)
        
        idx = np.random.choice(range(0, vocab_size), p=y.ravel())
        indices.append(idx)
        
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        a_prev = a
        
        seed+=1
        counter+=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [7]:
np.random.seed(2)
_, n_a = 20, 100

Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

indices = sample(parameters, char_to_ix, 0)

print("Sampling:")
print("list of sampled indices:", indices)
print("list of sampled characters:", [ix_to_char[i] for i in indices])

Sampling:
list of sampled indices: [12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 17, 24, 1, 13, 0, 0]
list of sampled characters: ['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'q', 'x', 'a', 'm', '\n', '\n']


In [8]:
def optimize(X, Y, a_prev, parameters, learning_rate=0.01):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    gradients = clip(gradients, 5)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]

In [9]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate=0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

Loss = 126.50397572165382
gradients["dWaa"][1][2] = 0.1947093153471637
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032002977
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]


In [10]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    n_x , n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open('datasets/dinos.txt') as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.seed(0)
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        Y = X[1:] + [char_to_ix["\n"]]
        
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)

        loss = smooth(loss, curr_loss)
        if (j % 2000 == 0):
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1
                
            print('\n')
            
    return parameters

In [11]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.087336

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


Iteration: 2000, Loss: 27.884160

Liusskeomnolxeros
Hmdaairus
Hytroligoraurus
Lecalosapaus
Xusicikoraurus
Abalpsamantisaurus
Tpraneronxeros


Iteration: 4000, Loss: 25.901815

Mivrosaurus
Inee
Ivtroplisaurus
Mbaaisaurus
Wusichisaurus
Cabaselachus
Toraperlethosdarenitochusthiamamumamaon


Iteration: 6000, Loss: 24.608779

Onwusceomosaurus
Lieeaerosaurus
Lxussaurus
Oma
Xusteonosaurus
Eeahosaurus
Toreonosaurus


Iteration: 8000, Loss: 24.070350

Onxusichepriuon
Kilabersaurus
Lutrodon
Omaaerosaurus
Xutrcheps
Edaksoje
Trodiktonus


Iteration: 10000, Loss: 23.844446

Onyusaurus
Klecalosaurus
Lustodon
Ola
Xusodonia
Eeaeosaurus
Troceosaurus


Iteration: 12000, Loss: 23.291971

Onyxosaurus
Kica
Lustrepiosaurus
Olaagrraiansaurus
Yuspangosaurus
Eealosaurus
Trognesaurus


Iteration: 14000, Loss: 23.382338

Meutromodromurus
Inda
Iutroinatorsaurus
Maca
Yusteratop

In [12]:
parameters = model(data, ix_to_char, char_to_ix, num_iterations=50000, n_a=25)

Iteration: 0, Loss: 23.087338

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


Iteration: 2000, Loss: 28.119757

Miwtroligoraurus
Iola
Ixtroligoraurus
Mca
Xusddlerolxeros
Ba
Troligoraurus


Iteration: 4000, Loss: 25.884315

Mivrosaurus
Inedalosaurus
Ivrosaurus
Medalosaurus
Xusibilosaurus
Cabbosaurus
Trolonlonus


Iteration: 6000, Loss: 24.742789

Onxtromonlonus
Jieeahropedosaurus
Kustongeronus
On
Xusokongonus
Eeajron
Trolonlonus


Iteration: 8000, Loss: 24.279739

Mivstolongaluroskereonos
Inecalton
Ivstolongakurosocrus
Mecajton
Xusperatops
Ecaisope
Usnerborav


Iteration: 10000, Loss: 24.021364

Liuscong
Inda
Ivrosaurus
Lecagosaurus
Xuspeodon
Caadosaurus
Usodon


Iteration: 12000, Loss: 23.444570

Ngxusoeonirax
Inca
Ivrosaurus
Ncaadosaurus
Wuspenatopthosaurus
Eeagosaurus
Uspenatopthosaurus


Iteration: 14000, Loss: 23.490056

Leustololoraviraterapros
Inecaltoledosaurus
Ivusthogshaurus
Lacaltoledosaurus
Wurodomus
Caacosaurus
Us

In [13]:
parameters = model(data, ix_to_char, char_to_ix, n_a=100)

Iteration: 0, Loss: 23.087337

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


Iteration: 2000, Loss: 27.694284

Mivusheomorcwhprudrhentoclusuroceletcen
Hmcabersalaosaurusateixalos
Iwtpraperonweros
Mbaalosanassaurostonaxanosaurus
Xushciiromxgortariphus
Baalpsalanthrgenwmegycerta
Tpsaurus


Iteration: 4000, Loss: 25.949507

Mivrosaurus
Ingcalosaurus
Ivrosaurus
Madamosaurus
Xuslerasaurus
Baberpia
Tosaurus


Iteration: 6000, Loss: 24.796461

Opussteneronverosaurus
Lieealosaurus
Lyusleolosaurus
Ondalrondbosaurusauriyelosaurus
Wusreolosaurus
Eeagosaurus
Trogolosaurus


Iteration: 8000, Loss: 24.327294

Onyusherasaurus
Klecalosaurus
Lyusidon
Olaados
Wuspconmolups
Edaeropa
Trochesaurus


Iteration: 10000, Loss: 24.047050

Mewusicheoodurosaurus
Inecalosaurus
Iusqraodoravesaurus
Macaerola
Vusrbiapodus
Adadosaurus
Trocekiocriosaurus


Iteration: 12000, Loss: 23.462616

Onxusieoinosoprosaurus
Ingaakosaurus
Kvosonorgosunosaurus
Oma
Vrpsau